In [1]:
# Data processing
# ==============================================================================

import pandas as pd
import numpy as np

# Data plotting
# ==============================================================================

import matplotlib.pyplot as plt
import seaborn as sns

# Modelling
# ==============================================================================

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from scipy.optimize import linprog
from imblearn.over_sampling import SMOTE

# Memory Optimization
# ==============================================================================

import gc
gc.collect()

In [2]:
# Load the datasets
# ==============================================================================
 
fct_mailers = pd.read_csv('~/OpenDoor/data/fct_mailers.csv')
dim_homes = pd.read_csv('~/OpenDoor/data/dim_homes.csv')
fct_offers = pd.read_csv('~/OpenDoor/data/fct_offers.csv')

# Remove duplicates from fct_offers
# fct_offers.drop_duplicates(inplace=True)

In [3]:
# Sanity check the see how the join between fct_mailers, fct_offers tables looks like
# ==============================================================================
# 
mergedf = pd.merge(fct_mailers, fct_offers, on='home_id', how='left')
mergedf.sort_values(by='home_id')

,mail_id,home_id,mail_date,offer_id,offer_date,offer_amount,offer_accepted
79773,ID14929213,3,2023-08-01T00:00:00.000Z,NaN,NaN,NaN,NaN
59912,ID18104728,24,2023-06-01T00:00:00.000Z,5c5e6b9d-bb09-48f8-80fe-886cd6891aa2,2023-07-11T00:00:00.000+0000,1.024258e+06,False
22880,ID92334873,61,2023-03-01T00:00:00.000Z,NaN,NaN,NaN,NaN
53352,ID93474536,83,2023-06-01T00:00:00.000Z,NaN,NaN,NaN,NaN
73746,ID41220770,107,2023-08-01T00:00:00.000Z,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
25590,ID98818355,2499895,2023-03-01T00:00:00.000Z,64f1bb21-df7b-467c-9cb6-b00f26a3e13c,2023-03-28T00:00:00.000+0000,1.354977e+06,False
101024,ID90904078,2499897,2023-11-01T00:00:00.000Z,NaN,NaN,NaN,NaN
36725,ID43219461,2499935,2023-04-01T00:00:00.000Z,NaN,NaN,NaN,NaN
23351,ID74918697,2499974,2023-03-01T00:00:00.000Z,NaN,NaN,NaN,NaN


We notice that the shape of this join and dim_homes are close

In [25]:
# sanity check
# ==============================================================================

mergedf['home_id'].nunique() # 120000 is the original number in the dataset

117399

In [4]:
# Create a new variable 'time_diff' from columns 'mail_date' and 'offer_date'
# ==============================================================================

mergedf['mail_date'] = pd.to_datetime(mergedf['mail_date'], format='%Y-%m-%dT%H:%M:%S.%fZ', utc=True)
mergedf['offer_date'] = pd.to_datetime(mergedf['offer_date'], format='%Y-%m-%dT%H:%M:%S.%f%z')
mergedf['time_diff'] = mergedf['offer_date'] - mergedf['mail_date']

In [5]:
# create the new dataset by merging the tables on home_id 
# ==============================================================================

mergedf1 = pd.merge(dim_homes, pd.merge(fct_mailers, fct_offers, on='home_id', how='left'),on='home_id', how='left')
mergedf1.sort_values(by='home_id')

,home_id,market,home_type,est_home_value,quality_score,home_size_sqft,nbr_bedrooms,nbr_bathrooms,construction_year,last_transaction_year,last_transaction_value,owner_name,est_owner_age,mail_id,mail_date,offer_id,offer_date,offer_amount,offer_accepted
0,0,New York,townhome,1.155426e+06,3,2519,5,3,2009,2023,1.155426e+06,Allison Hill,51,NaN,NaN,NaN,NaN,NaN,NaN
1,1,New York,townhome,7.186668e+05,83,1548,3,3,1992,1992,2.012799e+05,Noah Rhodes,42,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Phoenix,townhome,3.131741e+05,50,1164,1,1,1997,1997,6.477227e+04,Angie Henderson,21,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Dallas-Ft. Worth,single-family detached,2.042423e+06,61,3975,4,2,1982,2010,1.622118e+06,Daniel Wagner,50,ID14929213,2023-08-01T00:00:00.000Z,NaN,NaN,NaN,NaN
4,4,Dallas-Ft. Worth,townhome,7.201744e+05,51,1150,2,3,2015,2015,5.537206e+05,Cristian Santos,29,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2502645,2499995,San Francisco-Oakland-San Jose,single-family detached,1.023970e+06,91,2562,2,1,2011,2011,8.167252e+05,Dr. Heather Young,57,NaN,NaN,NaN,NaN,NaN,NaN
2502646,2499996,Detroit,single-family detached,3.850527e+05,8,1570,4,3,2008,2012,3.027382e+05,Paige Rivera,64,NaN,NaN,NaN,NaN,NaN,NaN
2502647,2499997,Atlanta,townhome,1.484337e+06,11,2463,4,3,2002,2002,5.551246e+05,Lisa Mcclure,36,NaN,NaN,NaN,NaN,NaN,NaN
2502648,2499998,Detroit,townhome,1.006164e+06,70,4430,3,4,1971,1971,9.627137e+04,Rachel Mcgee,56,NaN,NaN,NaN,NaN,NaN,NaN


Notice that the shape after the join is increased multifold. As expected, there are also a lot of NaN values. 

In [6]:
# create 3 new features based on the given features for logical way of buying a house
# ==============================================================================

# Calculate home age
mergedf1['home_age'] = 2024 - mergedf1['construction_year']

# Calculate years since last transaction
mergedf1['years_since_last_transaction'] = 2024 - mergedf1['last_transaction_year']

# Calculate home value per square foot
mergedf1['value_per_sqft'] = mergedf1['est_home_value'] / mergedf1['home_size_sqft']

I will now create the train and test datasets. Train dataset is the part of mergedf1 which has offer_accepted as Boolean ( True or False ). 

It will of course be a smaller set and the challenge here is having a huge test dataset which is obtained from the part of mergedf1 where offer_accepted is NaN. 

In [7]:
# Drop rows where column 'offer_accepted' has NaN values to get the train dataset
# ==============================================================================


df_cleaned = mergedf1.dropna(subset=['offer_accepted'])
df_cleaned.shape

(7482, 22)

In [8]:
# Sanity check 
# ==============================================================================

df_cleaned.columns

Index(['home_id', 'market', 'home_type', 'est_home_value', 'quality_score',
       'home_size_sqft', 'nbr_bedrooms', 'nbr_bathrooms', 'construction_year',
       'last_transaction_year', 'last_transaction_value', 'owner_name',
       'est_owner_age', 'mail_id', 'mail_date', 'offer_id', 'offer_date',
       'offer_amount', 'offer_accepted', 'home_age',
       'years_since_last_transaction', 'value_per_sqft'],
      dtype='object')

In [9]:
# Sanity check 
# ==============================================================================

df_cleaned.describe()

,home_id,est_home_value,quality_score,home_size_sqft,nbr_bedrooms,nbr_bathrooms,construction_year,last_transaction_year,last_transaction_value,est_owner_age,offer_amount,home_age,years_since_last_transaction,value_per_sqft
count,7.482000e+03,7.482000e+03,7482.000000,7482.000000,7482.000000,7482.000000,7482.000000,7482.000000,7.482000e+03,7482.000000,7.482000e+03,7482.000000,7482.000000,7482.000000
mean,1.240379e+06,8.316833e+05,32.744453,2530.856188,3.194333,2.369420,1991.369420,2002.120021,4.864075e+05,44.346298,7.913742e+05,32.630580,21.879979,349.705710
std,7.259970e+05,3.419252e+05,23.874531,1123.053474,1.402417,1.125403,18.309584,15.152816,3.223459e+05,12.415455,3.194095e+05,18.309584,15.152816,120.941001
min,2.400000e+01,1.053729e+05,0.000000,800.000000,1.000000,1.000000,1960.000000,1961.000000,5.553416e+03,20.000000,1.016781e+05,1.000000,1.000000,121.959333
25%,6.064770e+05,5.542370e+05,13.000000,1585.000000,2.000000,1.000000,1975.000000,1992.000000,2.261013e+05,34.000000,5.340941e+05,17.000000,9.000000,248.031103
50%,1.239022e+06,8.192200e+05,28.500000,2375.000000,3.000000,2.000000,1991.000000,2005.000000,4.241234e+05,43.000000,7.829246e+05,33.000000,19.000000,319.511830
75%,1.875454e+06,1.103317e+06,49.000000,3371.000000,4.000000,3.000000,2007.000000,2015.000000,6.904213e+05,54.000000,1.039572e+06,49.000000,32.000000,438.773335
max,2.499895e+06,1.499473e+06,100.000000,4995.000000,5.000000,4.000000,2023.000000,2023.000000,1.498547e+06,70.000000,1.436328e+06,64.000000,63.000000,715.509778


In [10]:
# Get the rows where column 'offer_accepted' has NaN values to get the test dataset
# ==============================================================================

dropped_rows = mergedf1[mergedf1['offer_accepted'].isna()]
dropped_rows.shape
                        

(2495168, 22)

In [11]:
# Sanity check 
# ==============================================================================

dropped_rows.columns

Index(['home_id', 'market', 'home_type', 'est_home_value', 'quality_score',
       'home_size_sqft', 'nbr_bedrooms', 'nbr_bathrooms', 'construction_year',
       'last_transaction_year', 'last_transaction_value', 'owner_name',
       'est_owner_age', 'mail_id', 'mail_date', 'offer_id', 'offer_date',
       'offer_amount', 'offer_accepted', 'home_age',
       'years_since_last_transaction', 'value_per_sqft'],
      dtype='object')

In [12]:
# Sanity check 
# ==============================================================================

dropped_rows['market'].nunique()

11

In [13]:
# Sanity check 
# ==============================================================================

dropped_rows['home_type'].nunique()

3

In [14]:
# Sanity check 
# ==============================================================================

dropped_rows.describe()

,home_id,est_home_value,quality_score,home_size_sqft,nbr_bedrooms,nbr_bathrooms,construction_year,last_transaction_year,last_transaction_value,est_owner_age,offer_amount,home_age,years_since_last_transaction,value_per_sqft
count,2.495168e+06,2.495168e+06,2.495168e+06,2.495168e+06,2.495168e+06,2.495168e+06,2.495168e+06,2.495168e+06,2.495168e+06,2.495168e+06,0.0,2.495168e+06,2.495168e+06,2.495168e+06
mean,1.249994e+06,1.130332e+06,5.002193e+01,2.899610e+03,2.998613e+00,2.499590e+00,1.986465e+03,2.000729e+03,6.498204e+05,4.501785e+01,NaN,3.753546e+01,2.327077e+01,3.905869e+02
std,7.216799e+05,6.082933e+05,2.914400e+01,1.212666e+03,1.414595e+00,1.118064e+00,2.136280e+01,1.599196e+01,4.960027e+05,1.472312e+01,NaN,2.136280e+01,1.599196e+01,1.255434e+02
min,0.000000e+00,4.219532e+04,0.000000e+00,8.000000e+02,1.000000e+00,1.000000e+00,1.950000e+03,1.960000e+03,1.500000e+03,2.000000e+01,NaN,1.000000e+00,1.000000e+00,5.045925e+01
25%,6.250278e+05,6.534461e+05,2.500000e+01,1.850000e+03,2.000000e+00,1.000000e+00,1.968000e+03,1.989000e+03,2.679871e+05,3.200000e+01,NaN,1.900000e+01,1.000000e+01,2.771625e+02
50%,1.249970e+06,1.019855e+06,5.000000e+01,2.899000e+03,3.000000e+00,2.000000e+00,1.986000e+03,2.003000e+03,5.241323e+05,4.500000e+01,NaN,3.800000e+01,2.100000e+01,3.736723e+02
75%,1.874952e+06,1.504828e+06,7.500000e+01,3.950000e+03,4.000000e+00,3.000000e+00,2.005000e+03,2.014000e+03,9.140218e+05,5.800000e+01,NaN,5.600000e+01,3.500000e+01,5.051794e+02
max,2.499999e+06,3.377521e+06,1.000000e+02,5.000000e+03,5.000000e+00,4.000000e+00,2.023000e+03,2.023000e+03,3.310148e+06,7.000000e+01,NaN,7.400000e+01,6.400000e+01,8.640407e+02


In [16]:
# Sanity check 
# ==============================================================================
 
dropped_rows['offer_amount'].value_counts()

Series([], Name: count, dtype: int64)

In [17]:
# Sanity check 
# ==============================================================================

dropped_rows['offer_amount'] = 0
## doing this to keep the feature in play - it has no variance - is infact harmful for the predictions but we will keep it 
## can use other methods like KNN to interpolate but I choose this due  to time constriant 

/var/folders/mh/crzr9fvs1hz479vjn820n9kc0000gq/T/ipykernel_46291/1427976220.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dropped_rows['offer_amount'] = 0


In [18]:
# Sanity check 
# ==============================================================================

dropped_rows['offer_amount'].value_counts()

offer_amount
0    2495168
Name: count, dtype: int64

In [19]:
# Exclude columns that are not needed for model fitting and scale the columns 
# ==============================================================================

columns_to_exclude = ['home_id', 'offer_id', 'offer_date', 'owner_name', 'construction_year', 'last_transaction_year', 'mail_id', 'mail_date']
X = df_cleaned.drop(columns=columns_to_exclude + ['offer_accepted'])

# Encode categorical variables
X = pd.get_dummies(X, columns=['home_type', 'market'])

# Identify and exclude columns causing issues during scaling
try:
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
except ValueError as e:
    print(e)
    problematic_columns = [col for col in X.columns if X[col].isnull().all()]
    print("Columns with 0 samples:", problematic_columns)
    X = X.drop(columns=problematic_columns)
    X_scaled = scaler.fit_transform(X)

y = df_cleaned['offer_accepted']

In [20]:

columns_to_exclude = ['home_id', 'offer_id', 'offer_date', 'owner_name', 'construction_year', 'last_transaction_year', 'mail_id', 'mail_date']
X = dropped_rows.drop(columns=columns_to_exclude + ['offer_accepted'])

# Encode categorical variables
X = pd.get_dummies(X, columns=['home_type', 'market'])

# Identify and exclude columns causing issues during scaling
try:
    scaler = StandardScaler()
    X_all_scaled = scaler.fit_transform(X)
except ValueError as e:
    print(e)
    problematic_columns = [col for col in X.columns if X[col].isnull().all()]
    print("Columns with 0 samples:", problematic_columns)
    X = X.drop(columns=problematic_columns)
    X_all_scaled = scaler.fit_transform(X)


I will use only constriant-based optimisation along with XGBoost classifier to get the top 10000 home_id to send fliers to 

In [23]:
# Constraint-Based Optimization to get the top 10000 homes to send fliers to 
# ==============================================================================

model_constraint = xgb.XGBClassifier(random_state=42)
model_constraint.fit(X_scaled, y)
probs_constraint = model_constraint.predict_proba(X_all_scaled)[:, 1]
c = -probs_constraint

# First, create A_eq_market
market_counts = dim_homes['market'].value_counts(normalize=True) * 10000
A_eq_market = np.array([(dim_homes['market'] == market).astype(int) for market in market_counts.index])
A_eq_market = np.vstack(A_eq_market)  # To ensure A_eq_market is a 2D array

# Get the number of columns in A_eq_market
num_cols = A_eq_market.shape[1]

# Create A_eq with the correct number of columns
A_eq = np.ones((1, num_cols))
b_eq = [10000]

b_eq_market = market_counts.values
A_eq = np.vstack([A_eq, A_eq_market])
b_eq = np.concatenate([b_eq, b_eq_market])

# Ensure the length of c matches the number of columns in A_eq
c = np.concatenate([c, np.zeros(num_cols - len(c))])

result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=(0, 1), method='highs')
selected_indices = np.where(result.x > 0.5)[0]
selected_home_ids = dim_homes.iloc[selected_indices]['home_id'].tolist()
print(selected_home_ids)
probs_constraint_optimized = np.zeros(len(dim_homes))
probs_constraint_optimized[selected_indices] = probs_constraint[selected_indices]

# Define weights for each method
weight_constraint = 1

# Compute weighted sum of predictions
weighted_probs = (weight_constraint * probs_constraint_optimized)

# Add weighted probabilities to the original dataframe
dim_homes['weighted_prob'] = weighted_probs

# Rank homes by weighted probability and select top 10,000
top_homes_weighted = dim_homes.sort_values(by='weighted_prob', ascending=False).head(10000)

# Extract the home IDs for the selected homes
selected_home_ids_weighted = top_homes_weighted['home_id'].tolist()
print(selected_home_ids_weighted)

[372, 599, 802, 970, 989, 1070, 1160, 1164, 2903, 3140, 3205, 3209, 3227, 3268, 3332, 3438, 4053, 4167, 4433, 4595, 4682, 4990, 5598, 5794, 5819, 5991, 6034, 6158, 6197, 6651, 6990, 7442, 7609, 7621, 7888, 8668, 8782, 9091, 9137, 9345, 9467, 9606, 9857, 10653, 10703, 10944, 11483, 11855, 12669, 13014, 13181, 13185, 13560, 13562, 13583, 13852, 14403, 14442, 14727, 14989, 15288, 15318, 15452, 16583, 16655, 16931, 17135, 17171, 17375, 17606, 17637, 17719, 17737, 18304, 18515, 19117, 19249, 19269, 19403, 20037, 20217, 20655, 20678, 20949, 21530, 21593, 21630, 21674, 22003, 22538, 22582, 22629, 22858, 23791, 24098, 24339, 24386, 24395, 24480, 24764, 25032, 25309, 25322, 25481, 25663, 25908, 26698, 26799, 26823, 27429, 27534, 28351, 28446, 28479, 28786, 28822, 28993, 29396, 29524, 30022, 30095, 30133, 30192, 30323, 30585, 30600, 30756, 30769, 31087, 31257, 31700, 31792, 32073, 32333, 32385, 32454, 32462, 32703, 32858, 33227, 33518, 33840, 34045, 34068, 34129, 34525, 34776, 35145, 35769, 3581

In [24]:
# Sanity check 
# ==============================================================================

type(selected_home_ids_weighted)

list

To evalute I check the est_home_value of the dim_homes table 

In [37]:
# # Extract rows where values in column 'home_id' match the list of values
# ==============================================================================

extracted_rows = dim_homes[dim_homes['home_id'].isin(selected_home_ids_weighted)]
extracted_rows

,home_id,market,home_type,est_home_value,quality_score,home_size_sqft,nbr_bedrooms,nbr_bathrooms,construction_year,last_transaction_year,last_transaction_value,owner_name,est_owner_age,weighted_prob
372,372,New York,single-family detached,7.254766e+05,71,1548,4,2,1985,1985,1.909682e+05,Ellen Morgan,21,0.835491
599,599,Houston,single-family detached,6.310070e+05,31,2596,5,3,2009,2009,3.564340e+05,Susan Thompson,33,0.796783
802,802,Detroit,townhome,1.119917e+06,19,4521,4,4,1997,1997,2.623034e+05,Reginald Morrow,58,0.803469
970,970,San Francisco-Oakland-San Jose,single-family detached,1.552382e+06,98,4173,1,3,1984,1984,9.816484e+05,Nicholas Rush,55,0.714376
989,989,New York,single-family detached,1.188438e+06,38,2561,3,1,1997,1997,5.037286e+05,Karen Johnson,40,0.852720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2493606,2493606,Houston,single-family detached,6.827155e+05,48,2581,4,2,1998,2023,6.827155e+05,Kathleen Miller,70,0.734197
2493655,2493655,New York,single-family detached,5.299110e+05,78,873,4,4,2008,2008,3.838903e+05,Tamara Anderson,35,0.698011
2493801,2493801,Detroit,single-family detached,9.080292e+05,65,3386,2,3,1991,1991,2.927498e+05,Jorge Blanchard PhD,35,0.740087
2494530,2494530,San Francisco-Oakland-San Jose,single-family detached,1.335721e+06,88,4147,1,2,1982,2006,7.213292e+05,Peter Ford,69,0.758621


In [34]:
# Sanity check 
# ==============================================================================

extracted_rows.describe()

,home_id,est_home_value,quality_score,home_size_sqft,nbr_bedrooms,nbr_bathrooms,construction_year,last_transaction_year,last_transaction_value,est_owner_age,offer_amount,home_age,years_since_last_transaction,value_per_sqft
count,9.976000e+03,9.976000e+03,9976.000000,9976.00000,9976.000000,9976.000000,9976.000000,9976.000000,9.976000e+03,9976.000000,9976.0,9976.000000,9976.000000,9976.000000
mean,1.254273e+06,1.128156e+06,49.805634,2900.29180,2.992382,2.488673,1986.394647,2000.890136,6.514500e+05,44.967221,0.0,37.605353,23.109864,390.115974
std,7.212893e+05,6.033853e+05,28.991310,1210.04678,1.412846,1.113990,21.370280,15.963328,4.934040e+05,14.732646,0.0,21.370280,15.963328,125.618895
min,3.720000e+02,7.867270e+04,0.000000,801.00000,1.000000,1.000000,1950.000000,1960.000000,5.267842e+03,20.000000,0.0,1.000000,1.000000,84.232010
25%,6.299075e+05,6.553812e+05,25.000000,1855.00000,2.000000,1.000000,1968.000000,1990.000000,2.660343e+05,32.000000,0.0,19.000000,10.000000,276.562472
50%,1.263987e+06,1.023392e+06,50.000000,2886.50000,3.000000,2.000000,1986.000000,2004.000000,5.311811e+05,45.000000,0.0,38.000000,20.000000,373.325640
75%,1.876760e+06,1.499888e+06,75.000000,3943.25000,4.000000,3.000000,2005.000000,2014.000000,9.203520e+05,58.000000,0.0,56.000000,34.000000,503.920938
max,2.494539e+06,3.212976e+06,100.000000,5000.00000,5.000000,4.000000,2023.000000,2023.000000,3.064113e+06,70.000000,0.0,74.000000,64.000000,724.357945


In [38]:
# Get the first 10000 values of the offered homes to compare the price  
# ==============================================================================

non_offered_homes = dim_homes[~dim_homes['home_id'].isin(selected_home_ids_weighted)]
non_offered_homes.sort_values(by='home_id')
non_offered_homes


,home_id,market,home_type,est_home_value,quality_score,home_size_sqft,nbr_bedrooms,nbr_bathrooms,construction_year,last_transaction_year,last_transaction_value,owner_name,est_owner_age,weighted_prob
0,0,New York,townhome,1.155426e+06,3,2519,5,3,2009,2023,1.155426e+06,Allison Hill,51,0.0
1,1,New York,townhome,7.186668e+05,83,1548,3,3,1992,1992,2.012799e+05,Noah Rhodes,42,0.0
2,2,Phoenix,townhome,3.131741e+05,50,1164,1,1,1997,1997,6.477227e+04,Angie Henderson,21,0.0
3,3,Dallas-Ft. Worth,single-family detached,2.042423e+06,61,3975,4,2,1982,2010,1.622118e+06,Daniel Wagner,50,0.0
4,4,Dallas-Ft. Worth,townhome,7.201744e+05,51,1150,2,3,2015,2015,5.537206e+05,Cristian Santos,29,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499995,2499995,San Francisco-Oakland-San Jose,single-family detached,1.023970e+06,91,2562,2,1,2011,2011,8.167252e+05,Dr. Heather Young,57,0.0
2499996,2499996,Detroit,single-family detached,3.850527e+05,8,1570,4,3,2008,2012,3.027382e+05,Paige Rivera,64,0.0
2499997,2499997,Atlanta,townhome,1.484337e+06,11,2463,4,3,2002,2002,5.551246e+05,Lisa Mcclure,36,0.0
2499998,2499998,Detroit,townhome,1.006164e+06,70,4430,3,4,1971,1971,9.627137e+04,Rachel Mcgee,56,0.0


In [39]:
# Get the first 10000 values of the non_offered homes to compare the price  
# ==============================================================================
 
non_offered_homes_10000 = non_offered_homes.head(10000).reset_index(drop=True)
non_offered_homes_10000

,home_id,market,home_type,est_home_value,quality_score,home_size_sqft,nbr_bedrooms,nbr_bathrooms,construction_year,last_transaction_year,last_transaction_value,owner_name,est_owner_age,weighted_prob
0,0,New York,townhome,1.155426e+06,3,2519,5,3,2009,2023,1.155426e+06,Allison Hill,51,0.0
1,1,New York,townhome,7.186668e+05,83,1548,3,3,1992,1992,2.012799e+05,Noah Rhodes,42,0.0
2,2,Phoenix,townhome,3.131741e+05,50,1164,1,1,1997,1997,6.477227e+04,Angie Henderson,21,0.0
3,3,Dallas-Ft. Worth,single-family detached,2.042423e+06,61,3975,4,2,1982,2010,1.622118e+06,Daniel Wagner,50,0.0
4,4,Dallas-Ft. Worth,townhome,7.201744e+05,51,1150,2,3,2015,2015,5.537206e+05,Cristian Santos,29,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,10038,Phoenix,single-family detached,1.087975e+06,91,3985,3,3,1998,1998,5.017612e+05,Kelly Brooks,52,0.0
9996,10039,San Francisco-Oakland-San Jose,single-family detached,9.201824e+05,82,2452,3,3,1985,1985,2.295091e+05,Christina Ferguson,61,0.0
9997,10040,Dallas-Ft. Worth,single-family detached,1.925071e+06,80,3122,2,4,2020,2020,1.679001e+06,Lauren Lopez,70,0.0
9998,10041,New York,townhome,1.934779e+06,99,4444,3,1,1992,1997,1.107736e+06,Danielle Burke,20,0.0


In [40]:
# Calculate mean, median, and standard deviation for column 'C' in non_offered_homes_10000 and 
# ==============================================================================

mean_c = non_offered_homes_10000['est_home_value'].mean()
median_c = non_offered_homes_10000['est_home_value'].median()
std_c = non_offered_homes_10000['est_home_value'].std()

# Calculate mean, median, and standard deviation for column 'D' in extracted_rows
mean_d = extracted_rows['est_home_value'].mean()
median_d = extracted_rows['est_home_value'].median()
std_d = extracted_rows['est_home_value'].std()

# Print the results
print("Column 'est_home_value' in non_offered_homes_10000:")
print(f"Mean: {mean_c}")
print(f"Median: {median_c}")
print(f"Standard Deviation: {std_c}")

print("\nColumn 'est_home_value' in extracted_rows:")
print(f"Mean: {mean_d}")
print(f"Median: {median_d}")
print(f"Standard Deviation: {std_d}")

Column 'est_home_value' in non_offered_homes_10000:
Mean: 1131627.0014026773
Median: 1014190.4305043472
Standard Deviation: 611063.6623271866

Column 'est_home_value' in extracted_rows:
Mean: 1127224.2472371946
Median: 1022992.2859720383
Standard Deviation: 602869.4489288601


We see that the tendencies of predicted values and actual values are close enough to accept the optimisation done using the above methodlogy. 